In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [25]:
num_samples = 1000
a = torch.randint(1, 1001, (num_samples, 1)).float()
b = torch.randint(1, 1001, (num_samples, 1)).float()

In [26]:
a_mean, a_std = a.mean(), a.std()
b_mean, b_std = b.mean(), b.std()

def normalize_data(a, b):
    a_normalized = (a - a_mean) / a_std
    b_normalized = (b - b_mean) / b_std
    return a_normalized, b_normalized

a_normalized, b_normalized = normalize_data(a, b)

In [27]:
input_features = torch.cat((a_normalized, b_normalized), dim=1)
target = torch.sqrt(a**2 + b**2)

In [28]:
dataset = TensorDataset(input_features, target)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(2, 64)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [30]:
model = NeuralNetwork()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{1000}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 6895.3965
Epoch [200/1000], Loss: 134.8116
Epoch [300/1000], Loss: 31.2078
Epoch [400/1000], Loss: 4.9897
Epoch [500/1000], Loss: 1.1150
Epoch [600/1000], Loss: 0.3533
Epoch [700/1000], Loss: 0.0995
Epoch [800/1000], Loss: 0.0643
Epoch [900/1000], Loss: 0.0125
Epoch [1000/1000], Loss: 0.2008


In [31]:
def absolute_percentage_error(actual, predicted):
    if actual == 0:
        return 0
    ape = abs((actual - predicted) / actual) * 100
    return ape

In [32]:
with torch.no_grad():
    test_a = torch.tensor([[40.0]])
    test_b = torch.tensor([[50.0]])

    # Normalize the test data
    test_a_normalized = (test_a - a_mean) / a_std
    test_b_normalized = (test_b - b_mean) / b_std

    test_inputs = torch.cat([test_a_normalized, test_b_normalized], dim=1)

    predicted = model(test_inputs)

    # Since we didn't normalize the output, no need to denormalize the prediction
    actual = torch.sqrt(test_a**2 + test_b**2)

    predicted_np = predicted.numpy()[0][0]
    actual_np = actual.numpy()[0][0]
    print(f"Predicted Square Root: {predicted_np:.4f}, True Square Root: {actual_np:.4f}")
    error = absolute_percentage_error(predicted_np, actual_np)
    print(f"Absolute Percentage Error: {error:.2f}%")

Predicted Square Root: 64.1262, True Square Root: 64.0312
Absolute Percentage Error: 0.15%
